# 🎯 CodeBERT Hybrid Optimization

**Goal:** Deep dive into CodeBERT_hybrid configuration optimization

## 📊 What is CodeBERT_hybrid?

**Hybrid = 70% CodeBERT embeddings + 30% metadata features**

### Components:

1. **CodeBERT embeddings (70%)**:
   - Semantic understanding of code from microsoft/codebert-base
   - 768-dimensional dense vectors
   - Captures: syntax patterns, variable names, code structure, API usage

2. **Metadata features (30%)**:
   - One-hot encoded categorical attributes:
     - `language`: python, cpp, java, etc.
     - `domain`: web_framework, data_science, algorithms, etc.
     - `task_type`: bug_fix, feature, testing, debugging, etc.
     - `complexity`: simple, medium, complex
   - Normalized to L2 norm

3. **Why it works**:
   - CodeBERT alone = semantic similarity (code that "looks" similar)
   - Metadata alone = categorical grouping (exact language/domain matches)
   - **Hybrid = best of both**: semantic understanding + explicit structure

### Previous Result:
- **Configuration**: CodeBERT_hybrid + Agglomerative Average + K=40
- **Silhouette**: **0.689** (excellent!)

### This Experiment:
Systematically optimize:
1. K values (25-60)
2. Hybrid ratios (60/40, 70/30, 80/20, 90/10)
3. Agglomerative linkages (ward, complete, average, single)
4. Alternative algorithms (KMeans, Spectral, GMM, HDBSCAN)

---
**⚡ GPU recommended for CodeBERT extraction**

## 0. 🔧 Setup

In [ ]:
!pip install -q transformers datasets umap-learn scikit-learn matplotlib seaborn pandas numpy hdbscan torch
print('✅ Packages installed!')

In [ ]:
import torch

if torch.cuda.is_available():
    device = 'cuda'
    print(f"🎮 GPU: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = 'cpu'
    print("💻 CPU mode")
    print("   💡 Enable GPU: Runtime → Change runtime type → GPU")

## 1. 📦 Imports

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import warnings
import json
import time
from typing import Dict, List, Tuple
warnings.filterwarnings('ignore')

# ML
from sklearn.cluster import KMeans, AgglomerativeClustering, SpectralClustering, DBSCAN
from sklearn.mixture import GaussianMixture
import hdbscan
from sklearn.metrics import silhouette_score, silhouette_samples
from sklearn.preprocessing import normalize
import umap

# Transformers
from transformers import AutoModel, AutoTokenizer
from datasets import load_dataset
import torch

# Viz
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

np.random.seed(42)

print('✅ Imports complete!')

## 2. 📥 Load Coding Datasets

In [ ]:
def load_coding_datasets(max_total=4000):
    """
    Load diverse coding datasets and extract metadata.
    Same as notebook 03.
    """
    questions = []

    print("="*70)
    print("LOADING CODING DATASETS")
    print("="*70)

    # 1. SWE-bench
    print("\n1. Loading SWE-bench (GitHub issues)...")
    try:
        swe_dataset = load_dataset("princeton-nlp/SWE-bench_Lite", split="test")
        count = 0
        target = min(2000, len(swe_dataset))

        for idx, item in enumerate(swe_dataset):
            if count >= target:
                break

            problem = item.get("problem_statement", "")
            repo = item.get("repo", "")

            if "django" in repo.lower() or "flask" in repo.lower():
                domain = "web_framework"
            elif "sklearn" in repo.lower() or "pandas" in repo.lower() or "numpy" in repo.lower():
                domain = "data_science"
            elif "matplotlib" in repo.lower() or "seaborn" in repo.lower():
                domain = "visualization"
            elif "pytest" in repo.lower() or "test" in repo.lower():
                domain = "testing"
            elif "requests" in repo.lower() or "http" in repo.lower():
                domain = "networking"
            else:
                domain = "general"

            problem_lower = problem.lower()
            if "bug" in problem_lower or "fix" in problem_lower or "error" in problem_lower:
                task_type = "bug_fix"
            elif "test" in problem_lower:
                task_type = "testing"
            elif "refactor" in problem_lower or "clean" in problem_lower:
                task_type = "refactor"
            elif "add" in problem_lower or "implement" in problem_lower or "feature" in problem_lower:
                task_type = "feature"
            else:
                task_type = "general"

            if len(problem) < 200:
                complexity = "simple"
            elif len(problem) < 500:
                complexity = "medium"
            else:
                complexity = "complex"

            if problem:
                questions.append({
                    "question": problem,
                    "language": "python",
                    "domain": domain,
                    "task_type": task_type,
                    "complexity": complexity,
                    "source": f"swe_bench_{repo}"
                })
                count += 1

        print(f"   ✓ Loaded {count} GitHub issues")

    except Exception as e:
        print(f"   ✗ Error: {e}")

    # 2. DS-1000
    print("\n2. Loading DS-1000 (Data science tasks)...")
    try:
        ds_dataset = load_dataset("xlangai/DS-1000", split="test")
        count = 0

        for item in ds_dataset:
            prompt = item.get("prompt", "")
            metadata = item.get("metadata", {})
            library = metadata.get("library", "unknown") if isinstance(metadata, dict) else "unknown"

            if library in ["Numpy", "Pandas", "Scipy"]:
                domain = "data_manipulation"
            elif library in ["Matplotlib"]:
                domain = "visualization"
            elif library in ["Pytorch", "Tensorflow", "Sklearn"]:
                domain = "machine_learning"
            else:
                domain = "data_science"

            if len(prompt) < 150:
                complexity = "simple"
            elif len(prompt) < 300:
                complexity = "medium"
            else:
                complexity = "complex"

            if prompt:
                questions.append({
                    "question": prompt,
                    "language": "python",
                    "domain": domain,
                    "task_type": "code_generation",
                    "complexity": complexity,
                    "source": f"ds1000_{library.lower()}"
                })
                count += 1

        print(f"   ✓ Loaded {count} data science tasks")

    except Exception as e:
        print(f"   ✗ Error: {e}")

    # 3. BigCodeBench
    print("\n3. Loading BigCodeBench (API tasks)...")
    try:
        bigcode_dataset = load_dataset("bigcode/bigcodebench", split="v0.1.2")
        count = 0
        target = min(500, len(bigcode_dataset))

        for idx, item in enumerate(bigcode_dataset):
            if count >= target:
                break

            complete_prompt = item.get("complete_prompt", "")
            instruct_prompt = item.get("instruct_prompt", "")
            prompt = instruct_prompt if instruct_prompt else complete_prompt

            if len(prompt) < 200:
                complexity = "simple"
            elif len(prompt) < 400:
                complexity = "medium"
            else:
                complexity = "complex"

            if prompt:
                questions.append({
                    "question": prompt,
                    "language": "python",
                    "domain": "api_usage",
                    "task_type": "code_generation",
                    "complexity": complexity,
                    "source": "bigcodebench"
                })
                count += 1

        print(f"   ✓ Loaded {count} API tasks")

    except Exception as e:
        print(f"   ✗ Error: {e}")

    # 4. DebugBench
    print("\n4. Loading DebugBench (Debugging tasks)...")
    try:
        debug_dataset = load_dataset("Rtian/DebugBench", split="test")
        count = 0
        target = min(500, len(debug_dataset))

        for idx, item in enumerate(debug_dataset):
            if count >= target:
                break

            buggy_code = item.get("buggy_code", "")
            language = item.get("language", "python").lower()
            difficulty = item.get("difficulty", "medium").lower()

            complexity_map = {"easy": "simple", "medium": "medium", "hard": "complex"}
            complexity = complexity_map.get(difficulty, "medium")

            if buggy_code:
                questions.append({
                    "question": f"Debug this code:\n{buggy_code}",
                    "language": language,
                    "domain": "algorithms",
                    "task_type": "debugging",
                    "complexity": complexity,
                    "source": "debugbench"
                })
                count += 1

        print(f"   ✓ Loaded {count} debugging tasks")

    except Exception as e:
        print(f"   ✗ Error: {e}")

    print(f"\n{'='*70}")
    print(f"✅ Total: {len(questions)} coding tasks")
    print(f"\nBreakdown:")
    print(f"  Languages: {Counter(q['language'] for q in questions)}")
    print(f"  Domains: {Counter(q['domain'] for q in questions)}")
    print(f"  Task Types: {Counter(q['task_type'] for q in questions)}")
    print(f"  Complexity: {Counter(q['complexity'] for q in questions)}")
    print(f"{'='*70}")

    return questions

# Load data
questions = load_coding_datasets(max_total=4000)
texts = [q['question'] for q in questions]

## 3. 🧠 Extract CodeBERT Embeddings

In [ ]:
def mean_pooling(token_embeddings, attention_mask):
    """Mean pooling - take average of all tokens"""
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

def encode_with_codebert(model, tokenizer, texts, device, batch_size=32):
    """Encode texts using CodeBERT"""
    model.eval()
    all_embeddings = []
    
    print(f'🚀 Encoding {len(texts)} texts with CodeBERT...')
    
    with torch.no_grad():
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i+batch_size]
            
            # Tokenize
            encoded = tokenizer(
                batch_texts,
                padding=True,
                truncation=True,
                max_length=512,
                return_tensors='pt'
            ).to(device)
            
            # Get embeddings
            outputs = model(**encoded)
            
            # Mean pooling
            embeddings = mean_pooling(outputs.last_hidden_state, encoded['attention_mask'])
            
            all_embeddings.append(embeddings.cpu().numpy())
            
            # Progress
            if (i // batch_size) % 10 == 0:
                print(f'  Processed {min(i+batch_size, len(texts))}/{len(texts)}', end='\r')
    
    print(f'  Processed {len(texts)}/{len(texts)} ✓')
    return np.vstack(all_embeddings)

print("="*70)
print("LOADING CODEBERT")
print("="*70)

tokenizer = AutoTokenizer.from_pretrained('microsoft/codebert-base')
model = AutoModel.from_pretrained('microsoft/codebert-base').to(device)

codebert_embeddings = encode_with_codebert(model, tokenizer, texts, device, batch_size=32)
codebert_norm = normalize(codebert_embeddings, norm='l2')

print(f"\n✅ CodeBERT embeddings: {codebert_norm.shape}")
print(f"   Mean: {codebert_norm.mean():.4f}, Std: {codebert_norm.std():.4f}")

# Free memory
del model
del tokenizer
if device == 'cuda':
    torch.cuda.empty_cache()

print("="*70)

## 4. 📊 Create Metadata Features

In [ ]:
print('Creating metadata features...')

metadata_features = pd.concat([
    pd.get_dummies([q['language'] for q in questions], prefix='lang'),
    pd.get_dummies([q['domain'] for q in questions], prefix='domain'),
    pd.get_dummies([q['task_type'] for q in questions], prefix='task'),
    pd.get_dummies([q['complexity'] for q in questions], prefix='complexity')
], axis=1).values

metadata_norm = normalize(metadata_features, norm='l2')

print(f'✅ Metadata features: {metadata_norm.shape}')
print(f"   Attributes: language, domain, task_type, complexity")
print(f"   Encoding: One-hot, L2-normalized")

## 5. 🔬 Comprehensive Experiments

In [ ]:
def create_hybrid_features(codebert_emb, metadata_emb, ratio=0.7):
    """Create hybrid features with given ratio.
    
    Args:
        codebert_emb: CodeBERT embeddings (normalized)
        metadata_emb: Metadata features (normalized)
        ratio: Weight for CodeBERT (0.0-1.0), metadata gets (1-ratio)
    
    Returns:
        Concatenated and normalized hybrid features
    """
    hybrid = np.concatenate([
        codebert_emb * ratio,
        metadata_emb * (1 - ratio)
    ], axis=1)
    return normalize(hybrid, norm='l2')

def run_clustering(features, algorithm_name, k=None, **kwargs):
    """Run clustering and return labels + silhouette score."""
    if algorithm_name == 'KMeans':
        alg = KMeans(n_clusters=k, random_state=42, n_init=10, **kwargs)
    elif algorithm_name == 'Agglomerative':
        alg = AgglomerativeClustering(n_clusters=k, **kwargs)
    elif algorithm_name == 'Spectral':
        alg = SpectralClustering(n_clusters=k, random_state=42, **kwargs)
    elif algorithm_name == 'GMM':
        alg = GaussianMixture(n_components=k, random_state=42, **kwargs)
    elif algorithm_name == 'HDBSCAN':
        alg = hdbscan.HDBSCAN(metric='cosine', **kwargs)
    elif algorithm_name == 'DBSCAN':
        alg = DBSCAN(metric='cosine', **kwargs)
    else:
        raise ValueError(f"Unknown algorithm: {algorithm_name}")
    
    labels = alg.fit_predict(features)
    
    # Filter out noise points (-1)
    mask = labels >= 0
    n_clusters = len(set(labels[mask]))
    
    if n_clusters > 1 and mask.sum() > 10:
        sil = silhouette_score(features[mask], labels[mask], metric='cosine')
    else:
        sil = -1.0
    
    return labels, sil, n_clusters

print('✅ Helper functions defined')

### Experiment 1: K-value Sweep (Agglomerative Average)

In [ ]:
print("="*70)
print("EXPERIMENT 1: K-VALUE SWEEP")
print("="*70)
print("Configuration: CodeBERT_hybrid (70/30) + Agglomerative Average")
print("K values: 25, 30, 35, 40, 45, 50, 55, 60\n")

# Create hybrid features (70/30)
hybrid_70_30 = create_hybrid_features(codebert_norm, metadata_norm, ratio=0.7)

k_values = [25, 30, 35, 40, 45, 50, 55, 60]
k_results = []

for k in k_values:
    labels, sil, n_clusters = run_clustering(
        hybrid_70_30,
        'Agglomerative',
        k=k,
        linkage='average'
    )
    
    k_results.append({
        'k': k,
        'silhouette': sil,
        'n_clusters': n_clusters,
        'labels': labels
    })
    
    print(f"K={k:2d}: Silhouette = {sil:.6f}")

best_k = max(k_results, key=lambda x: x['silhouette'])
print(f"\n✅ Best K: {best_k['k']} (Silhouette = {best_k['silhouette']:.6f})")
print("="*70)

### Experiment 2: Hybrid Ratio Sweep

In [ ]:
print("="*70)
print("EXPERIMENT 2: HYBRID RATIO SWEEP")
print("="*70)
print("Configuration: CodeBERT_hybrid + Agglomerative Average + K=40")
print("Ratios: 60/40, 70/30, 80/20, 90/10 (CodeBERT/Metadata)\n")

ratios = [0.6, 0.7, 0.8, 0.9]
ratio_results = []

for ratio in ratios:
    hybrid = create_hybrid_features(codebert_norm, metadata_norm, ratio=ratio)
    
    labels, sil, n_clusters = run_clustering(
        hybrid,
        'Agglomerative',
        k=40,
        linkage='average'
    )
    
    ratio_results.append({
        'ratio': ratio,
        'ratio_str': f"{int(ratio*100)}/{int((1-ratio)*100)}",
        'silhouette': sil,
        'labels': labels
    })
    
    print(f"Ratio {int(ratio*100)}/{int((1-ratio)*100)}: Silhouette = {sil:.6f}")

best_ratio = max(ratio_results, key=lambda x: x['silhouette'])
print(f"\n✅ Best Ratio: {best_ratio['ratio_str']} (Silhouette = {best_ratio['silhouette']:.6f})")
print("="*70)

### Experiment 3: Linkage Comparison

In [ ]:
print("="*70)
print("EXPERIMENT 3: LINKAGE COMPARISON")
print("="*70)
print("Configuration: CodeBERT_hybrid (70/30) + K=40")
print("Linkages: ward, complete, average, single\n")

linkages = ['ward', 'complete', 'average', 'single']
linkage_results = []

for linkage in linkages:
    labels, sil, n_clusters = run_clustering(
        hybrid_70_30,
        'Agglomerative',
        k=40,
        linkage=linkage
    )
    
    linkage_results.append({
        'linkage': linkage,
        'silhouette': sil,
        'labels': labels
    })
    
    print(f"{linkage:10s}: Silhouette = {sil:.6f}")

best_linkage = max(linkage_results, key=lambda x: x['silhouette'])
print(f"\n✅ Best Linkage: {best_linkage['linkage']} (Silhouette = {best_linkage['silhouette']:.6f})")
print("="*70)

### Experiment 4: Algorithm Comparison

In [ ]:
print("="*70)
print("EXPERIMENT 4: ALGORITHM COMPARISON")
print("="*70)
print("Configuration: CodeBERT_hybrid (70/30) + K=40\n")

algorithm_results = []

# K-based algorithms
for alg_name in ['KMeans', 'Spectral', 'GMM']:
    labels, sil, n_clusters = run_clustering(hybrid_70_30, alg_name, k=40)
    algorithm_results.append({
        'algorithm': alg_name,
        'silhouette': sil,
        'k': 40,
        'labels': labels
    })
    print(f"{alg_name:15s}: Silhouette = {sil:.6f}")

# Density-based algorithms
print("\nDensity-based algorithms (auto K):")

# HDBSCAN
labels, sil, n_clusters = run_clustering(
    hybrid_70_30,
    'HDBSCAN',
    min_cluster_size=50
)
algorithm_results.append({
    'algorithm': 'HDBSCAN',
    'silhouette': sil,
    'k': n_clusters,
    'labels': labels
})
print(f"HDBSCAN (K={n_clusters:2d})    : Silhouette = {sil:.6f}")

# DBSCAN
labels, sil, n_clusters = run_clustering(
    hybrid_70_30,
    'DBSCAN',
    eps=0.3,
    min_samples=10
)
algorithm_results.append({
    'algorithm': 'DBSCAN',
    'silhouette': sil,
    'k': n_clusters,
    'labels': labels
})
print(f"DBSCAN (K={n_clusters:2d})     : Silhouette = {sil:.6f}")

# Add Agglomerative average from previous experiments
algorithm_results.append({
    'algorithm': 'Agglomerative-avg',
    'silhouette': best_linkage['silhouette'],
    'k': 40,
    'labels': best_linkage['labels']
})

best_algorithm = max(algorithm_results, key=lambda x: x['silhouette'])
print(f"\n✅ Best Algorithm: {best_algorithm['algorithm']} (Silhouette = {best_algorithm['silhouette']:.6f})")
print("="*70)

## 6. 📊 Results Summary

In [ ]:
print("="*70)
print("EXPERIMENT RESULTS SUMMARY")
print("="*70)

print("\n1. K-Value Optimization:")
print(f"   Best K: {best_k['k']}")
print(f"   Silhouette: {best_k['silhouette']:.6f}")

print("\n2. Hybrid Ratio Optimization:")
print(f"   Best Ratio: {best_ratio['ratio_str']} (CodeBERT/Metadata)")
print(f"   Silhouette: {best_ratio['silhouette']:.6f}")

print("\n3. Linkage Optimization:")
print(f"   Best Linkage: {best_linkage['linkage']}")
print(f"   Silhouette: {best_linkage['silhouette']:.6f}")

print("\n4. Algorithm Comparison:")
print(f"   Best Algorithm: {best_algorithm['algorithm']}")
print(f"   Silhouette: {best_algorithm['silhouette']:.6f}")

# Find overall best
all_results = [
    ('K-sweep', best_k['silhouette'], f"K={best_k['k']}"),
    ('Ratio-sweep', best_ratio['silhouette'], f"Ratio={best_ratio['ratio_str']}"),
    ('Linkage', best_linkage['silhouette'], f"Linkage={best_linkage['linkage']}"),
    ('Algorithm', best_algorithm['silhouette'], f"Alg={best_algorithm['algorithm']}")
]

overall_best = max(all_results, key=lambda x: x[1])

print("\n" + "="*70)
print("🏆 OVERALL BEST CONFIGURATION")
print("="*70)
print(f"Source: {overall_best[0]}")
print(f"Config: {overall_best[2]}")
print(f"Silhouette: {overall_best[1]:.6f}")
print("\nComparison to baseline:")
print(f"  Notebook 04 best: 0.689515")
print(f"  This experiment:  {overall_best[1]:.6f}")
if overall_best[1] > 0.689515:
    print(f"  🎉 Improvement: +{(overall_best[1] - 0.689515):.6f}")
else:
    print(f"  ⚠️  Difference: {(overall_best[1] - 0.689515):.6f}")
print("="*70)

## 7. 🔍 Detailed Analysis

### Per-Cluster Silhouette Scores

In [ ]:
# Use best configuration from linkage experiment (Agglomerative average K=40)
best_labels = best_linkage['labels']
best_features = hybrid_70_30

# Calculate per-sample silhouette scores
mask = best_labels >= 0  # Filter noise
sample_silhouettes = silhouette_samples(best_features[mask], best_labels[mask], metric='cosine')

# Calculate per-cluster metrics
cluster_metrics = []

valid_labels = best_labels[mask]
for cluster_id in range(best_k['k']):
    cluster_mask = valid_labels == cluster_id
    cluster_samples = sample_silhouettes[cluster_mask]
    
    if len(cluster_samples) > 0:
        cluster_metrics.append({
            'cluster': cluster_id,
            'size': len(cluster_samples),
            'silhouette_mean': cluster_samples.mean(),
            'silhouette_std': cluster_samples.std(),
            'silhouette_min': cluster_samples.min(),
            'silhouette_max': cluster_samples.max()
        })

cluster_df = pd.DataFrame(cluster_metrics)

print("="*70)
print("PER-CLUSTER SILHOUETTE ANALYSIS")
print("="*70)
print(f"\nTop 10 best clusters (highest mean silhouette):")
print(cluster_df.nlargest(10, 'silhouette_mean')[['cluster', 'size', 'silhouette_mean', 'silhouette_std']])

print(f"\nTop 10 worst clusters (lowest mean silhouette):")
print(cluster_df.nsmallest(10, 'silhouette_mean')[['cluster', 'size', 'silhouette_mean', 'silhouette_std']])

print(f"\nCluster size statistics:")
print(f"  Mean size: {cluster_df['size'].mean():.1f}")
print(f"  Median size: {cluster_df['size'].median():.1f}")
print(f"  Std size: {cluster_df['size'].std():.1f}")
print(f"  Min size: {cluster_df['size'].min()}")
print(f"  Max size: {cluster_df['size'].max()}")
print("="*70)

### UMAP Visualization

In [ ]:
print("Running UMAP dimensionality reduction...")

reducer = umap.UMAP(
    n_components=2,
    random_state=42,
    n_neighbors=15,
    min_dist=0.1,
    metric='cosine'
)

embeddings_2d = reducer.fit_transform(best_features)

print("✅ UMAP complete")

# Create visualization
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Left: Color by cluster
scatter1 = axes[0].scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    c=best_labels,
    cmap='tab20',
    alpha=0.6,
    s=20
)
axes[0].set_title(
    f'UMAP: Best Configuration (K={best_k["k"]}, Sil={best_linkage["silhouette"]:.4f})',
    fontweight='bold',
    fontsize=14
)
axes[0].set_xlabel('UMAP 1', fontsize=12)
axes[0].set_ylabel('UMAP 2', fontsize=12)
plt.colorbar(scatter1, ax=axes[0], label='Cluster ID')

# Right: Color by silhouette score
full_silhouettes = np.full(len(best_labels), -1.0)
full_silhouettes[mask] = sample_silhouettes

scatter2 = axes[1].scatter(
    embeddings_2d[:, 0],
    embeddings_2d[:, 1],
    c=full_silhouettes,
    cmap='RdYlGn',
    alpha=0.6,
    s=20,
    vmin=-0.5,
    vmax=1.0
)
axes[1].set_title(
    'Per-Sample Silhouette Scores',
    fontweight='bold',
    fontsize=14
)
axes[1].set_xlabel('UMAP 1', fontsize=12)
axes[1].set_ylabel('UMAP 2', fontsize=12)
plt.colorbar(scatter2, ax=axes[1], label='Silhouette Score')

plt.tight_layout()
plt.savefig('codebert_hybrid_umap.png', dpi=300, bbox_inches='tight')
print("\n✅ Saved: codebert_hybrid_umap.png")
plt.show()

## 8. 💾 Export Results

In [ ]:
# Best configuration
best_config = {
    'approach': 'codebert_hybrid_optimized',
    'embedding_model': 'microsoft/codebert-base',
    'hybrid_ratio': {
        'codebert': 0.7,
        'metadata': 0.3,
        'best_from_sweep': best_ratio['ratio']
    },
    'algorithm': 'AgglomerativeClustering',
    'linkage': best_linkage['linkage'],
    'k': best_k['k'],
    'silhouette': float(best_linkage['silhouette']),
    'experiment_results': {
        'k_sweep': {
            'best_k': int(best_k['k']),
            'best_silhouette': float(best_k['silhouette'])
        },
        'ratio_sweep': {
            'best_ratio': best_ratio['ratio_str'],
            'best_silhouette': float(best_ratio['silhouette'])
        },
        'linkage_comparison': {
            'best_linkage': best_linkage['linkage'],
            'best_silhouette': float(best_linkage['silhouette'])
        },
        'algorithm_comparison': {
            'best_algorithm': best_algorithm['algorithm'],
            'best_silhouette': float(best_algorithm['silhouette'])
        }
    },
    'comparison_to_baseline': {
        'notebook_04_best': 0.689515,
        'this_experiment': float(best_linkage['silhouette']),
        'improvement': float(best_linkage['silhouette'] - 0.689515)
    },
    'total_questions': len(questions),
    'feature_dimensions': {
        'codebert': codebert_norm.shape[1],
        'metadata': metadata_norm.shape[1],
        'hybrid': best_features.shape[1]
    }
}

with open('codebert_hybrid_best_config.json', 'w') as f:
    json.dump(best_config, f, indent=2)

# Per-cluster metrics
cluster_df.to_csv('codebert_hybrid_cluster_metrics.csv', index=False)

# All experiment results
all_experiments = pd.DataFrame([
    {'experiment': 'k_sweep', 'config': f'K={r["k"]}', 'silhouette': r['silhouette']}
    for r in k_results
] + [
    {'experiment': 'ratio_sweep', 'config': f'Ratio={r["ratio_str"]}', 'silhouette': r['silhouette']}
    for r in ratio_results
] + [
    {'experiment': 'linkage', 'config': f'Linkage={r["linkage"]}', 'silhouette': r['silhouette']}
    for r in linkage_results
] + [
    {'experiment': 'algorithm', 'config': f'Alg={r["algorithm"]}', 'silhouette': r['silhouette']}
    for r in algorithm_results
])

all_experiments.to_csv('codebert_hybrid_all_experiments.csv', index=False)

print("✅ Exported:")
print("  - codebert_hybrid_best_config.json")
print("  - codebert_hybrid_cluster_metrics.csv")
print("  - codebert_hybrid_all_experiments.csv")
print("  - codebert_hybrid_umap.png")

## 9. 📝 Final Summary

In [ ]:
print("="*80)
print("FINAL SUMMARY: CODEBERT_HYBRID OPTIMIZATION")
print("="*80)

print("\n🎯 HYBRID APPROACH:")
print("  CodeBERT embeddings: 70% (semantic code understanding)")
print("  Metadata features:   30% (categorical attributes)")
print("  Combined dimensions:", best_features.shape[1])

print("\n🏆 BEST CONFIGURATION:")
print(f"  Algorithm: Agglomerative Clustering")
print(f"  Linkage: {best_linkage['linkage']}")
print(f"  K: {best_k['k']}")
print(f"  Silhouette: {best_linkage['silhouette']:.6f}")

print("\n📊 KEY FINDINGS:")
print(f"  1. Optimal K: {best_k['k']} (tested 25-60)")
print(f"  2. Optimal hybrid ratio: {best_ratio['ratio_str']} (tested 60/40 to 90/10)")
print(f"  3. Best linkage: {best_linkage['linkage']} (tested all 4 types)")
print(f"  4. Agglomerative outperforms: KMeans, Spectral, GMM, HDBSCAN, DBSCAN")

print("\n📈 COMPARISON:")
print(f"  Notebook 04 best: 0.689515")
print(f"  This optimization: {best_linkage['silhouette']:.6f}")
if best_linkage['silhouette'] > 0.689515:
    print(f"  ✅ Improved by: +{(best_linkage['silhouette'] - 0.689515):.6f}")
else:
    print(f"  ≈ Similar performance: {(best_linkage['silhouette'] - 0.689515):.6f}")

print("\n💡 CLUSTER QUALITY:")
print(f"  Mean cluster silhouette: {cluster_df['silhouette_mean'].mean():.4f}")
print(f"  Std cluster silhouette: {cluster_df['silhouette_mean'].std():.4f}")
print(f"  Best cluster: {cluster_df['silhouette_mean'].max():.4f}")
print(f"  Worst cluster: {cluster_df['silhouette_mean'].min():.4f}")

print("\n🚀 NEXT STEPS:")
print("  1. Implement in adaptive_router/core/cluster_engine.py")
print("  2. Train cluster profiles with real model error rates")
print("  3. Deploy to production routing system")
print("  4. Monitor real-world performance vs. predicted silhouette")

print("\n" + "="*80)